#### 🥇 Gold Layer – Analytical & Business Tables

##### Purpose
The Gold layer contains aggregated and business-ready tables
used for dashboards, reporting, and decision-making.

##### Key Responsibilities
- Trend analysis
- Regional and country comparisons


- **Step - 1 Load Silver Enriched Data**

In [0]:
#loading silver tables
from pyspark.sql.functions import col
df_silver = spark.table("silver_co2_enriched")
df_silver_casted = (
    df_silver
        .withColumn("year", col("year").cast("int"))
        .withColumn("co2_emissions_mt", col("co2_emissions_mt").cast("double"))
        .withColumn("population", col("population").cast("long"))
        .withColumn("gdp_usd", col("gdp_usd").cast("double"))
        .withColumn("co2_per_capita", col("co2_per_capita").cast("double"))
)


In [0]:
# using default catalog and schema
spark.sql("USE CATALOG co2_emissions")
spark.sql("USE SCHEMA default")


DataFrame[]

- **Step 2 — Global Emissions Trend**

In [0]:
#global year-over-year analysis(TABLE_1)
from pyspark.sql.functions import sum
# Aggregate global CO2 emissions year-wise by summing total emissions
gold_global_yoy = (
    df_silver_casted
        .groupBy("year")
        .agg(
            sum("co2_emissions_mt").alias("total_co2_emissions_mt")
        )
)
gold_global_yoy.write.format("delta").mode("overwrite") \
    .saveAsTable("gold_global_yoy_emissions")


In [0]:
%sql
SELECT COUNT(*) FROM co2_emissions.default.gold_global_yoy_emissions;

COUNT(*)
30


In [0]:
%sql
DESCRIBE co2_emissions.default.gold_global_yoy_emissions;

col_name,data_type,comment
year,int,null
total_co2_emissions_mt,double,null


- **Step 3 — Country-Level Analysis**

In [0]:
#country-wise yearly emissions comparison(TABLE_2)
from pyspark.sql.functions import sum, avg
# Aggregate country-wise yearly CO2 emissions and average per-capita emissions for trend and comparison analysis
gold_country_yearly_emissions = (
    df_silver_casted
        .groupBy("country", "year")
        .agg(
            sum("co2_emissions_mt").alias("total_co2_emissions_mt"),
            avg("co2_per_capita").alias("avg_co2_per_capita")
        )
        .orderBy("country", "year")
)
gold_country_yearly_emissions.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("co2_emissions.default.gold_country_yearly_emissions")

print("Gold Table 2 created successfully")



Gold Table 2 created successfully


In [0]:
%sql
SELECT country, year, COUNT(*)
FROM co2_emissions.default.gold_country_yearly_emissions
GROUP BY country, year
HAVING COUNT(*) > 1;


country,year,COUNT(*)


- **Step 4 — Regional Summary**

In [0]:
#regional emissions comparision(TABLE_3)
from pyspark.sql.functions import sum, avg
# Aggregate total and average per-capita CO2 emissions at the regional level
gold_regional_emissions_summary = (
    df_silver_casted
        .groupBy("region")
        .agg(
            sum("co2_emissions_mt").alias("total_co2_emissions_mt"),
            avg("co2_per_capita").alias("avg_co2_per_capita")
        )
        .orderBy(col("total_co2_emissions_mt").desc())
)
gold_regional_emissions_summary.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("co2_emissions.default.gold_regional_emissions_summary")

print("Gold Table 3 created successfully")


Gold Table 3 created successfully


In [0]:
%sql
SELECT region, COUNT(*)
FROM co2_emissions.default.gold_regional_emissions_summary
GROUP BY region;


region,COUNT(*)
Asia,1
Europe,1
North America,1
South America,1
Oceania,1


In [0]:
%sql
DESCRIBE co2_emissions.default.gold_regional_emissions_summary;


col_name,data_type,comment
region,string,null
total_co2_emissions_mt,double,null
avg_co2_per_capita,double,null


- **Step 5 - High Emitting Regions**

In [0]:
#identifying high emission regions(TABLE_4)
from pyspark.sql.functions import sum
# Identify and rank regions by total CO2 emissions to highlight highest emitting regions
gold_high_emission_regions = (
    df_silver_casted
        .groupBy("region")
        .agg(
            sum("co2_emissions_mt").alias("total_co2_emissions_mt")
        )
        .orderBy(col("total_co2_emissions_mt").desc())
)
gold_high_emission_regions.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("co2_emissions.default.gold_high_emission_regions")
print("Gold Table 4 created successfully")


Gold Table 4 created successfully


In [0]:
%sql
SELECT *
FROM co2_emissions.default.gold_high_emission_regions;

region,total_co2_emissions_mt
Asia,1.5495172535999852E8
Europe,1.487794726100003E8
North America,9.880175875000048E7
South America,4.958119267999997E7
Oceania,4.9500418009999916E7


In [0]:
%sql
--consistency check
SELECT
  a.region,
  a.total_co2_emissions_mt AS from_table4,
  b.total_co2_emissions_mt AS from_table3
FROM co2_emissions.default.gold_high_emission_regions a
JOIN co2_emissions.default.gold_regional_emissions_summary b
ON a.region = b.region;


region,from_table4,from_table3
Asia,1.5495172535999852E8,1.5495172535999852E8
Europe,1.487794726100003E8,1.487794726100003E8
North America,9.880175875000048E7,9.880175875000048E7
South America,4.958119267999997E7,4.958119267999997E7
Oceania,4.9500418009999916E7,4.9500418009999916E7


- **Step 6 - Population corelation**

In [0]:
#corelation b/w population and emissions(TABLE_5)
from pyspark.sql.functions import corr
# Calculate the correlation between population and CO2 emissions at the country level
gold_population_emissions_corr = (
    df_silver_casted
        .groupBy("country")
        .agg(
            corr("population", "co2_emissions_mt")
            .alias("population_co2_emissions_correlation")
        )
)
gold_population_emissions_corr.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("co2_emissions.default.gold_population_emissions_correlation")

print("Gold Table 5 created successfully")


Gold Table 5 created successfully


- **Step 7 - Income level analysis**

In [0]:
#emission analysis based on income level(TABLE_6)
from pyspark.sql.functions import avg, sum
# Aggregate CO2 emissions and per-capita emissions by income level
gold_income_level_emissions = (
    df_silver_casted
        .groupBy("income_level")
        .agg(
            avg("co2_per_capita").alias("avg_co2_per_capita"),
            sum("co2_emissions_mt").alias("total_co2_emissions_mt")
        )
        .orderBy(col("total_co2_emissions_mt").desc())
)
gold_income_level_emissions.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("co2_emissions.default.gold_income_level_emissions")

print("Gold Table 6 created successfully")



Gold Table 6 created successfully


In [0]:
%sql
SELECT income_level, COUNT(*)
FROM co2_emissions.default.gold_income_level_emissions
GROUP BY income_level;


income_level,COUNT(*)
Lower-Middle,1
Low,1
Upper-Middle,1
High,1
Unknown,1


In [0]:
%sql
CREATE OR REPLACE TABLE gold_country_cluster_summary AS
-- Create a Gold table that clusters countries based on total CO2 emissions
SELECT
    country,
    region,
    total_co2_emissions_mt,
    -- Countries are classified as High Polluter, Moderate, or Eco-Friendly using percentile-based thresholds
    CASE
        WHEN total_co2_emissions_mt >= p66 THEN 'High Polluter'
        WHEN total_co2_emissions_mt >= p33 THEN 'Moderate'
        ELSE 'Eco-Friendly'
    END AS emission_cluster
FROM
(
    -- Aggregate total emissions per country
    SELECT
        country,
        region,
        SUM(co2_emissions_mt) AS total_co2_emissions_mt
    FROM silver_co2_enriched
    GROUP BY country, region
) country_totals
CROSS JOIN
(
    -- Percentile thresholds for clustering
    -- Compute 33rd and 66th percentile thresholds for clustering
    SELECT
        percentile_approx(total_co2_emissions_mt, 0.33) AS p33,
        percentile_approx(total_co2_emissions_mt, 0.66) AS p66
    FROM
    (
        SELECT
            SUM(co2_emissions_mt) AS total_co2_emissions_mt
        FROM silver_co2_enriched
        GROUP BY country
    )
) thresholds;




num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT
    emission_cluster,
    COUNT(*) AS country_count,
    MIN(total_co2_emissions_mt) AS min_val,
    MAX(total_co2_emissions_mt) AS max_val
FROM gold_country_cluster_summary
GROUP BY emission_cluster;


emission_cluster,country_count,min_val,max_val
Moderate,4,4.9438180740000054E7,4.954237030999992E7
High Polluter,4,4.958119267999997E7,5.0309012530000575E7
Eco-Friendly,3,7524273.160000002,4.8492746220000125E7


In [0]:
%sql
--Emission spike alert mechanism
CREATE OR REPLACE TABLE gold_emission_alerts AS
SELECT
    e.country,
    e.region,
    e.year,
    e.total_co2_emissions_mt,
    t.threshold_value,
    CASE
        WHEN e.total_co2_emissions_mt >= t.threshold_value
             THEN 'ALERT'
        ELSE 'NORMAL'
    END AS alert_status
FROM
(
    SELECT
        country,
        region,
        year,
        SUM(co2_emissions_mt) AS total_co2_emissions_mt
    FROM silver_co2_enriched
    GROUP BY country, region, year
) e
CROSS JOIN
(
    SELECT
        percentile_approx(total_co2_emissions_mt, 0.95) AS threshold_value
    FROM
    (
        SELECT
            SUM(co2_emissions_mt) AS total_co2_emissions_mt
        FROM silver_co2_enriched
        GROUP BY country, year
    )
) t;


num_affected_rows,num_inserted_rows


In [0]:
%sql
--verifying all tables
SHOW TABLES IN co2_emissions.default;


database,tableName,isTemporary
default,bronze_co2_emissions,false
default,dlt_bronze_co2,false
default,dlt_bronze_co2_emissions,false
default,dlt_gold_country_cluster_summary,false
default,dlt_gold_country_yearly_emissions,false
default,dlt_gold_emission_alerts,false
default,dlt_gold_global_yoy_emissions,false
default,dlt_gold_high_emission_regions,false
default,dlt_gold_income_level_emissions,false
default,dlt_gold_population_emissions_correlation,false
